# RAG

Retrieval-Augmented Generation (RAG) is a technique that combines the strengths of pre-trained language models with the ability to retrieve information from a large corpus of documents. RAG **enables the language model to produce more informed, accurate, and contextually relevant answers** than by relying on its pre-trained knowledge alone.

At Cohere, all RAG calls come with... **precise citations**! 🎉
The model cites which groups of words, in the RAG chunks, were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from and they help users gain visibility into the model reasoning.  

RAG consists of 3 steps:
- Step 1: Indexing and given a user query, retrieve the relevant chunks from the index
- Step 2: Optionally, rerank the retrieved chunks
- Step 3: Generate the model final answer with **precise citations**, given the retrieved and reranked chunks




## Step 0 - Imports & Getting some data

In this example, we'll use a recent piece of text, that wasn't in the training data: the Wikipedia page of the movie "Dune 2".   

In practice, you would typically do RAG on much longer text, that doesn't fit in the context window of the model.

In [1]:
# we'll use Cohere to cover all building blocks of RAG
# TODO: upgrade to "cohere>5"
%pip install "cohere<5" --quiet


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cohere
API_KEY = "..." # fill in your Cohere API key here
co = cohere.Client("VpapZVh1VY3ugmvhkV44ZcVsz3RTFB2zJilg8WYe")

In [ ]:
# we'll get some wikipedia data
# !pip install wikipedia --quiet
# import wikipedia

  Preparing metadata (setup.py) ... done


In [4]:
# let's get the wikipedia article about Dune Part Two
with open('sample-docs/esop/esop-sample-2.txt', 'r') as file:
    text = file.read().replace('\n', '')

print(f"The text has roughly {len(text.split())} words.")

The text has roughly 3262 words.


## Step 1 - Indexing and given a user query, retrieve the relevant chunks from the index

We index the document in a vector database. This requires getting the documents, chunking them, embedding, and indexing them in a vector database. Then we retrieved relevant results based on the users' query.

### We split the document into chunks of roughly 512 words

In [5]:
# For chunking let's use langchain to help us split the text
%pip install -qU langchain-text-splitters --quiet
from langchain_text_splitters import RecursiveCharacterTextSplitter

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Create basic configurations to chunk the text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# Split the text into chunks with some overlap
chunks_ = text_splitter.create_documents([text])
chunks = [c.page_content for c in chunks_]
print(f"The text has been broken down in {len(chunks)} chunks.")

The text has been broken down in 46 chunks.


### Embed every text chunk

Cohere embeddings are state-of-the-art.

In [8]:
# Because the texts being embedded are the chunks we are searching over, we set the input type as search_doc
model="embed-english-v3.0"
response = co.embed(
    texts= chunks,
    model=model,
    input_type="search_document",
    embedding_types=['float']
)
embeddings = response.embeddings.float
print(f"We just computed {len(embeddings)} embeddings.")

We just computed 46 embeddings.


### Store the embeddings in a vector database

We use the simplest vector database ever: a python dictionary using `np.array()`.

In [9]:
# We use the simplest vector database ever: a python dictionary
!pip install numpy --quiet

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
import numpy as np
vector_database = {i: np.array(embedding) for i, embedding in enumerate(embeddings)}
# { 0: array([...]), 1: array([...]), 2: array([...]), ..., 10: array([...]) }

## Given a user query, retrieve the relevant chunks from the vector database


### Define the user question

In [11]:
# query = "What is the name of the optionee? Respond only with the name and nothing else."
query = "Are the options ISO or NSO? Repond only with ISO or NSO and nothing else."

# Note: the relevant passage in the wikipedia page we're looking for is:
# "[...] Dune: Part Two was originally scheduled to be released on October 20, 2023, but was delayed to November 17, 2023, before moving forward two weeks to November 3, 2023, to adjust to changes in release schedules from other studios. It was later postponed by over four months to March 15, 2024, due to the 2023 Hollywood labor disputes. After the strikes were resolved, the film moved once more up two weeks to March 1, 2024. [...]"

### Embed the user question

Cohere embeddings are state-of-the-art.

In [12]:
# Because the text being embedded is the search query, we set the input type as search_query
response = co.embed(
    texts=[query],
    model=model,
    input_type="search_query",
    embedding_types=['float']
)
query_embedding = response.embeddings.float[0]
print("query_embedding: ", query_embedding)

query_embedding:  [-0.0362854, -0.0033473969, -0.002653122, -0.028717041, -0.031982422, 0.021118164, -0.019622803, 0.013130188, -0.015625, 0.020599365, -0.017700195, -0.01108551, -0.035858154, -0.02848816, 0.027191162, -0.03729248, 0.0012083054, -0.007671356, -0.013282776, -0.05178833, 0.02130127, -0.05706787, 0.025604248, 0.020141602, 0.023757935, 0.04736328, 0.008430481, -0.015014648, 0.036499023, 0.011627197, 0.019546509, -0.048431396, 0.0020256042, 0.027832031, -0.02444458, 0.031204224, -0.005088806, -0.004623413, -0.012825012, 0.018585205, 0.024429321, -0.01184845, 0.044189453, -0.0033683777, -0.015487671, 0.026000977, -0.014579773, -0.037109375, 0.020523071, 0.029754639, -0.012825012, -0.022476196, 0.01612854, -0.059173584, 0.0031108856, -0.0016956329, -0.030944824, 0.0071754456, 0.0026760101, 0.007827759, 0.015670776, 0.041992188, 0.021606445, 0.047851562, 0.047210693, -0.056488037, 0.062561035, 0.019699097, -0.024887085, 0.033203125, 0.0440979, 0.016738892, 0.020706177, -0.0880

### Retrieve the most relevant chunks from the vector database

We use cosine similarity to find the most similar chunks

In [13]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Calculate similarity between the user question & each chunk
similarities = [cosine_similarity(query_embedding, chunk) for chunk in embeddings]
print("similarity scores: ", similarities)

# Get indices of the top 10 most similar chunks
sorted_indices = np.argsort(similarities)[::-1]

# Keep only the top 10 indices
top_indices = sorted_indices[:10]
print("Here are the indices of the top 10 chunks after retrieval: ", top_indices)

# Retrieve the top 10 most similar chunks
top_chunks_after_retrieval = [chunks[i] for i in top_indices]
print("Here are the top 10 chunks after retrieval: ")
for t in top_chunks_after_retrieval:
    print("== " + t)

similarity scores:  [0.14608116519196138, 0.22674057625325622, 0.20311996519086137, 0.15956129122681317, 0.35395973878805964, 0.2140226145569584, 0.18235560537066342, 0.18398022103531095, 0.2047525322199158, 0.14790231696608075, 0.21717034205725197, 0.15366216639595587, 0.2250012355985921, 0.23834481226899934, 0.20862891296394914, 0.25510111220110065, 0.20479051897849504, 0.16822052722031697, 0.24709635878776082, 0.20493648626132785, 0.3029409416862092, 0.3518498605908433, 0.2398414197832897, 0.21209605184803837, 0.1770955901417053, 0.22771108532677484, 0.21109573559885142, 0.3785508440231242, 0.13061456525571583, 0.17293050395203302, 0.15736756043237954, 0.2140388696262672, 0.25233739099620933, 0.2786217590606114, 0.2313459814973475, 0.1392863723724924, 0.2134192160624585, 0.24563221555190315, 0.2629577229314225, 0.22432679223968527, 0.255530331336756, 0.1878416418819459, 0.18850428435789915, 0.10807399357403387, 0.11319260762321336, 0.02641185238857361]
Here are the indices of the to

## Step 2 - Rerank the chunks retrieved from the vector database

We rerank the 10 chunks retrieved from the vector database. Reranking boosts retrieval accuracy.

Reranking lets us go from 10 chunks retrieved from the vector database, to the 3 most relevant chunks.

In [14]:
response = co.rerank(
    query=query,
    documents=top_chunks_after_retrieval,
    top_n=3,
    model="rerank-english-v2.0",
)

top_chunks_after_rerank = [result.document['text'] for result in response]
print("Here are the top 3 chunks after rerank: ")
for t in top_chunks_after_rerank:
    print("== " + t)

Here are the top 3 chunks after rerank: 
== (the "Optionee") an option (the "Option") to purchase the number of Shares, as set forth in the Notice of Stock Option Grant, at the exercise price per Share set forth in the Notice of Stock Option Grant (the "Exercise Price"), subject to the terms and conditions of this Option Agreement and the Plan. This Option is intended to be a Nonstatutory Stock Option ("NSO") or an Incentive Stock Option ("ISO"), as provided in the Notice of Stock Option Grant.      2.2    Exercise of Option.
== that otherwise would be issued to the Optionee pursuant to the exercise of this Option. The Company may refuse to honor the exercise and refuse to deliver Shares if such withholding amounts are not delivered at the time of exercise.            (B)    Notice of Disqualifying Disposition of ISO Shares. If the Option is an ISO, and if the Optionee sells or otherwise disposes of any of the Shares acquired pursuant to the exercise of the ISO on or before the later o

## Step 3 - Generate the model final answer, given the retrieved and reranked chunks

In [15]:
# preamble containing instructions about the task and the desired style for the output.
preamble = """
## Task & Context
You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

In [16]:
# retrieved documents
documents = [
    {"title": "chunk 0", "snippet": top_chunks_after_rerank[0]},
    {"title": "chunk 1", "snippet": top_chunks_after_rerank[1]},
    {"title": "chunk 2", "snippet": top_chunks_after_rerank[2]},
  ]

# get model response
response = co.chat(
  message=query,
  documents=documents,
  preamble=preamble,
  model="command-r",
  temperature=0.3
)

print("Final answer:")
print(response.text)

Final answer:
ISO.


Note: this is indeed the answer you'd expect, and here was the passage of text in wikipedia explaining it!

" [...] Dune: Part Two was originally scheduled to be released on October 20, 2023, but was delayed to November 17, 2023, before moving forward two weeks to November 3, 2023, to adjust to changes in release schedules from other studios. It was later postponed by over four months to March 15, 2024, due to the 2023 Hollywood labor disputes. After the strikes were resolved, the film moved once more up two weeks to March 1, 2024. [...]"

## Bonus: Citations come for free with Cohere! 🎉

At Cohere, all RAG calls come with... precise citations! 🎉
The model cites which groups of words, in the RAG chunks, were used to generate the final answer.  
These citations make it easy to check where the model’s generated response claims are coming from.  
They help users gain visibility into the model reasoning, and sanity check the final model generation.  
These citations are optional — you can decide to ignore them.


In [ ]:
print("Citations that support the final answer:")
for cite in response.citations:
  print(cite)

Citations that support the final answer:
{'start': 63, 'end': 79, 'text': 'Denis Villeneuve', 'document_ids': ['doc_0']}
{'start': 81, 'end': 102, 'text': 'director and producer', 'document_ids': ['doc_0']}
{'start': 105, 'end': 116, 'text': 'Jon Spaihts', 'document_ids': ['doc_0']}
{'start': 118, 'end': 145, 'text': 'co-writer of the screenplay', 'document_ids': ['doc_0']}
{'start': 148, 'end': 159, 'text': 'Mary Parent', 'document_ids': ['doc_1']}
{'start': 164, 'end': 175, 'text': 'Cale Boyter', 'document_ids': ['doc_1']}
{'start': 177, 'end': 186, 'text': 'producers', 'document_ids': ['doc_1']}
{'start': 190, 'end': 204, 'text': 'Tanya Lapointe', 'document_ids': ['doc_1']}
{'start': 206, 'end': 219, 'text': 'Brian Herbert', 'document_ids': ['doc_1']}
{'start': 221, 'end': 234, 'text': 'Byron Merritt', 'document_ids': ['doc_1']}
{'start': 236, 'end': 247, 'text': 'Kim Herbert', 'document_ids': ['doc_1']}
{'start': 249, 'end': 260, 'text': 'Thomas Tull', 'document_ids': ['doc_1']}
{'

In [ ]:
def insert_citations_in_order(text, citations):
    """
    A helper function to pretty print citations.
    """
    offset = 0
    document_id_to_number = {}
    citation_number = 0
    modified_citations = []

    # Process citations, assigning numbers based on unique document_ids
    for citation in citations:
        citation_numbers = []
        for document_id in sorted(citation["document_ids"]):
            if document_id not in document_id_to_number:
                citation_number += 1  # Increment for a new document_id
                document_id_to_number[document_id] = citation_number
            citation_numbers.append(document_id_to_number[document_id])

        # Adjust start/end with offset
        start, end = citation['start'] + offset, citation['end'] + offset
        placeholder = ''.join([f'[{number}]' for number in citation_numbers])
        # Bold the cited text and append the placeholder
        modification = f'**{text[start:end]}**{placeholder}'
        # Replace the cited text with its bolded version + placeholder
        text = text[:start] + modification + text[end:]
        # Update the offset for subsequent replacements
        offset += len(modification) - (end - start)

    # Prepare citations for listing at the bottom, ensuring unique document_ids are listed once
    unique_citations = {number: doc_id for doc_id, number in document_id_to_number.items()}
    citation_list = '\n'.join([f'[{doc_id}] source: "{documents[doc_id - 1]["snippet"]}"' for doc_id, number in sorted(unique_citations.items(), key=lambda item: item[1])])
    text_with_citations = f'{text}\n\n{citation_list}'

    return text_with_citations


print(insert_citations_in_order(response.text, response.citations))


Here are the key crew members involved in 'Dune: Part Two':

- **Denis Villeneuve**[1]: **director and producer**[1]
- **Jon Spaihts**[1]: **co-writer of the screenplay**[1]
- **Mary Parent**[2] and **Cale Boyter**[2]: **producers**[2] 
- **Tanya Lapointe**[2], **Brian Herbert**[2], **Byron Merritt**[2], **Kim Herbert**[2], **Thomas Tull**[2], **Richard P. Rubinstein**[2], **John Harrison**[2], **Herbert W. Gain**[2] and **Kevin J. Anderson**[2]: **executive producers**[2] 
- **Joe Walker**[3]: **editor**[3]
- **Brad Riker**[3]: **supervising art director**[3]
- **Patrice Vermette**[3]: **production designer**[3]
- **Paul Lambert**[3]: **visual effects supervisor**[3]
- **Gerd Nefzer**[3]: **special effects supervisor**[3]
- **Thomas Struthers**[3]: **stunt coordinator.**[3] 

A number of crew members from the first film returned for the sequel, which is set to be released in **2024.**[1]

[1] source: "Dune: Part Two is a 2024 American epic science fiction film directed and produced by

In [89]:
def evaluate(query: str, context: list[str], generated_answer: str) -> int:
   cohere_client = cohere.Client(API_KEY)
   structured_prompt = "This is the provided context: \n"
   for chunk in context:
       structured_prompt += chunk + "\n==========\n"
   structured_prompt += f"This is the user query: {query}\n"
   structured_prompt += "\n==========\n" + f"This is the answer to be evaluated: {generated_answer}"
  
   response = co.chat(
       preamble="You are an evaluation bot that evaluates the answers to questions given the context used to answer that question. The context used to answer that question will be extracted from financial documents. Please answer with 'True' if the provided answer correctly answers the question and is grounded in the provided context. Answer false if the provided answer does not match the question or is not grounded in the context",
       message=structured_prompt
   )
   print(response)
   print(response.text)
   return eval(response.text)


def get_data(path, fields):
  with open(path, 'r') as file:
    text = file.read().replace('\n', '')

  print(f"The text has roughly {len(text.split())} words.")


  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
  )

  # Split the text into chunks with some overlap
  chunks_ = text_splitter.create_documents([text])
  chunks = [c.page_content for c in chunks_]

  model="embed-english-v3.0"
  response = co.embed(
      texts= chunks,
      model=model,
      input_type="search_document",
      embedding_types=['float']
  )
  embeddings = response.embeddings.float
  print(f"We just computed {len(embeddings)} embeddings.")
  
  vector_database = {i: np.array(embedding) for i, embedding in enumerate(embeddings)}

  results = {}

  for field in fields:
    query = f"{field['query']}. It may be described as {', '.join(field['keywords'])}. It will appear in the document in the form of {field['in_document_type']}"
    response = co.embed(
      texts=[query],
      model=model,
      input_type="search_query",
      embedding_types=['float']
    )
    query_embedding = response.embeddings.float[0]
    print("query_embedding: ", query_embedding)

    similarities = [cosine_similarity(query_embedding, chunk) for chunk in embeddings]
    print("similarity scores: ", similarities)

    # Get indices of the top 10 most similar chunks
    sorted_indices = np.argsort(similarities)[::-1]

    # Keep only the top 10 indices
    top_indices = sorted_indices[:10]
    print("Here are the indices of the top 10 chunks after retrieval: ", top_indices)

    # Retrieve the top 10 most similar chunks
    top_chunks_after_retrieval = [chunks[i] for i in top_indices]
    print("Here are the top 10 chunks after retrieval: ")
    for t in top_chunks_after_retrieval:
        print("== " + t)

    response = co.rerank(
        query=query,
        documents=top_chunks_after_retrieval,
        top_n=3,
        model="rerank-english-v2.0",
    )

    top_chunks_after_rerank = [result.document['text'] for result in response]
    print("Here are the top 3 chunks after rerank: ")
    for t in top_chunks_after_rerank:
        print("== " + t)
    
    preamble = f"""
    ## Task & Context
    You are searching for information in a large document. Extract only the requested piece of information.

    ## Style Guide
    Your response must be of type {field['response_type']}. Do not respond in a sentence. Respond only with the data and nothing else. Do not round numbers and maintain decimal precision.
    """

    documents = [
      {"title": "chunk 0", "snippet": top_chunks_after_rerank[0]},
      {"title": "chunk 1", "snippet": top_chunks_after_rerank[1]},
      {"title": "chunk 2", "snippet": top_chunks_after_rerank[2]},
    ]

    # get model response
    response = co.chat(
      message=query,
      documents=documents,
      preamble=preamble,
      model="command-r",
      temperature=0.3
    )

    print("Final answer:")
    print(response.text)

    evaluator_approved = evaluate(query, top_chunks_after_rerank, response.text)

    results[field['name']] = { 'response': response.text, 'evaluator_approved': evaluator_approved }
    
  
  return results




In [92]:
option_data_fields = [
  {
    "name": "grantee_name",
    "query": "What is the name of the grantee? ",
    "keywords": [
      "grantee name",
      "optionee name",
      "employee name",
    ],
    "examples": [
      "John Doe",
      "Jane Smith",
    ],
    "in_document_type": "a person's name",
    "response_type": "string",
  },
  {
    "name": "vesting_start_date",
    "query": "What is the vesting start date (if present)? ",
    "examples": [
      "April 10, 2022",
      "04/10/2022",
    ],
    "keywords": [
      "vesting start date",
      "vesting start backdate",
    ],
    "in_document_type": "a date in human readable format",
    "response_type": "ISO 8601 date format",
  },
  {
    "name": "exercise_price",
    "query": "What is the exercise price? It may be fractions of cents. Maintain precision in your response.",
    "keywords": [
      "exercise price",
      "price per share",
      "strike price",
    ],
    "examples": [
      "$0.0001",
      "$42",
      "ten dollars",
    ],
    "in_document_type": "a number or money value",
    "response_type": "floating point",
  },
  {
    "name": "board_approval_date",
    "query": "What is the board approval date?",
    "keywords": [
      "board approval date",
      "approval date",
    ],
    "in_document_type": "a date in human readable format",
    "response_type": "ISO 8601 date format",
  },
  {
    "name": "grant_date",
    "query": "What is the grant date?",
    "keywords": [
      "grant date",
      "issue date",
    ],
    "in_document_type": "a date in human readable format",
    "response_type": "ISO 8601 date format",
  },
    {
    "name": "iso_nso",
    "query": "Is the option type ISO or NSO?",
    "keywords": [
      "option type",
      "Incentive stock option",
      "Non-qualified stock option",
    ],
    "in_document_type": "ISO or NSO",
    "response_type": "only the string 'ISO' or 'NSO'",
  },
]

In [93]:
extracted_data = get_data('sample-docs/esop/esop-sample-2.txt', option_data_fields)
print(extracted_data)

The text has roughly 3261 words.
We just computed 46 embeddings.
query_embedding:  [-0.045654297, -0.023468018, -0.047210693, 0.0003039837, -0.05050659, -0.016433716, -0.058166504, 0.033447266, 0.016235352, 0.021652222, 0.0032234192, 0.009132385, 0.018447876, -0.029647827, 0.028625488, 0.00015640259, 0.023147583, 0.047424316, 0.042297363, 0.017242432, 0.007434845, 0.056793213, -0.059509277, -0.039855957, 0.061065674, -0.03741455, -0.023269653, -0.010276794, 0.06237793, 0.021911621, 0.026931763, -0.013313293, 0.031585693, -0.004524231, 0.0026607513, 0.083740234, 0.0047950745, 0.0018548965, 0.020095825, -0.002702713, 0.028671265, -0.028869629, -0.03353882, 0.008163452, -0.05706787, -0.0068626404, 0.025405884, 0.015174866, -0.009117126, 0.07183838, 0.008728027, 0.04168701, 0.0019388199, 0.086364746, 0.017181396, 0.035705566, -0.0154953, 0.02130127, 0.012626648, -0.027923584, -0.026626587, 0.013946533, 0.028198242, 0.032104492, -0.0009012222, -0.012084961, 0.014533997, -0.0054740906, 0.041